In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.onnx
import torch.jit

# Verificar si se dispone de una GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


: 

In [3]:
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image, UnidentifiedImageError

class PlantDataset(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Mapeo de nombres de carpetas a etiquetas numéricas

        self.class_map = {
            'achira': 0,
            'calanchoe': 1,
            'diente_de_leon': 2,
            'flor_de_rosa': 3,
            'palta': 4
        }

        # Cargar todas las rutas de imágenes y sus etiquetas
        for root_dir in root_dirs:
            class_name = root_dir.split('/')[-1].lower()  # Asumimos que el nombre de la carpeta es la clase
            label = self.class_map[class_name]  # Obtener el label correspondiente
            for root, _, files in os.walk(root_dir):
                for file_name in files:
                    if file_name.endswith(('.png', '.jpg', '.jpeg')):
                        file_path = os.path.join(root, file_name)
                        self.image_paths.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except UnidentifiedImageError as e:
            print(f"Error: No se pudo identificar la imagen en {img_path}. {e}")
            return None, label  # Puedes manejar esto en tu DataLoader para omitir imágenes no válidas




In [4]:
# Rutas a las carpetas descomprimidas
root_dirs = [
    '/content/drive/MyDrive/IA 2/Dataset/descomprimido/achira',
    '/content/drive/MyDrive/IA 2/Dataset/descomprimido/calanchoe',
    '/content/drive/MyDrive/IA 2/Dataset/diente_de_leon',
    '/content/drive/MyDrive/IA 2/Dataset/descomprimido/flor_de_rosa',
    '/content/drive/MyDrive/IA 2/Dataset/descomprimido/palta'
]

In [5]:
# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
from torch.utils.data import DataLoader

# Definir un collate_fn personalizado que filtra los valores None
def collate_fn(batch):
    # Filtrar los ejemplos que contienen None
    batch = list(filter(lambda x: x[0] is not None, batch))

    # Si el batch queda vacío después de filtrar, devuelve un tensor vacío
    if len(batch) == 0:
        return torch.empty(0), torch.empty(0)

    # Usar el collate por defecto para el resto
    return torch.utils.data.dataloader.default_collate(batch)



In [7]:

# Crear dataset y dataloader
dataset = PlantDataset(root_dirs, transform=transform)
# Crear el DataLoader con el collate_fn personalizado
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, prefetch_factor=2, collate_fn=collate_fn)




#dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, prefetch_factor=2, collate_fn=collate_fn)

#dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, prefetch_factor=2)

#dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
import torch
from torch.utils.data import Subset

# Asumiendo que 'dataset' es tu dataset original
subset_indices = torch.randperm(len(dataset))[:int(0.6* len(dataset))]
subset_dataset = Subset(dataset, subset_indices)

# Crea un nuevo DataLoader usando el subset
subset_dataloader = DataLoader(subset_dataset, batch_size=64, shuffle=True, num_workers=4, prefetch_factor=2)

In [9]:
# Prueba con un DataLoader más simple
dataloader_simple = DataLoader(subset_dataset, batch_size=64, shuffle=True)
subset_size_simple = len(subset_dataset)
print(f'Cantidad de datos en el subconjunto con DataLoader simple: {subset_size_simple}')


Cantidad de datos en el subconjunto con DataLoader simple: 48028


In [10]:
total_images = len(dataset)
print(f'Cantidad total de imágenes en el dataset: {total_images}')


Cantidad total de imágenes en el dataset: 80047


In [11]:
# Obtener y mostrar la cantidad de datos en el subconjunto
subset_size = len(subset_dataset)
print(f'Cantidad de datos en el subconjunto: {subset_size}')

Cantidad de datos en el subconjunto: 48028


In [12]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Mapeo de etiquetas a nombres de clases
index_to_class = {0: 'achira', 1: 'calanchoe', 2: 'diente_de_leon', 3: 'flor_de_rosa', 4: 'palta'}

# Función para mostrar imágenes
def imshow(img, label):
    img = img / 2 + 0.5  # desnormalizar la imagen
    npimg = img.numpy()
    class_name = index_to_class.get(label.item(), 'desconocido')  # Obtener nombre de clase
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(f'Etiqueta: {class_name}')
    plt.show()

# Iterar sobre el DataLoader
data_iter = iter(dataloader)
images, labels = next(data_iter)

# Mostrar imágenes en un batch
for i in range(len(images)):
    imshow(images[i], labels[i])


Output hidden; open in https://colab.research.google.com to view.

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(128 * 128 * 3, 50)
        self.fc2 = nn.Linear(50, 5)

    def forward(self, x):
        x = x.view(-1, 128 * 128 * 3)  # Aplanar las imágenes
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MLP()


In [ ]:

import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

# Definir el optimizador y la función de pérdida
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Configurar el entrenamiento
epochs = 130
checkpoint_interval = 5
train_losses = []

# Función para calcular la precisión
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    corrects = (preds == labels).sum().item()
    accuracy = corrects / len(labels)
    return accuracy

# Cargar el estado del modelo y el optimizador desde el checkpoint
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    return start_epoch

# Verificar si existe un checkpoint y cargarlo
checkpoint_path = '/content/drive/MyDrive/IA 2/Checkpoints/mlp_checkpoint_latest.pth'
start_epoch = 0
if os.path.isfile(checkpoint_path):
    print("Cargando checkpoint...")
    start_epoch = load_checkpoint(checkpoint_path)
    print(f"Resumir desde la época {start_epoch}")

Cargando checkpoint...


In [ ]:
import os

def verify_dataset(image_paths):
    missing_files = [path for path in image_paths if not os.path.exists(path)]
    if missing_files:
        print(f"Archivos faltantes: {len(missing_files)}")
        for file in missing_files:
            print(file)
    else:
        print("Todos los archivos están presentes.")

# Llama a esta función antes de iniciar el entrenamiento
verify_dataset(dataset.image_paths)


In [16]:
from PIL import Image
import torch
import os

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = [path for path in image_paths if os.path.exists(path)]  # Filtrar archivos inexistentes
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform is not None:
                image = self.transform(image)
            label = self.labels[index]
            return image, label
        except FileNotFoundError:
            print(f"Archivo no encontrado: {img_path}. Saltando a la siguiente imagen.")
            return None, None

    def __len__(self):
        return len(self.image_paths)


In [17]:
# Inicializar lista para guardar las imágenes que no se identificaron
imagenes_no_identificadas = []

# Entrenamiento
for epoch in range(start_epoch, epochs):
    model.train()
    running_loss = 0.0
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    # Barra de progreso para el dataloader
    for batch_idx, (images, labels) in enumerate(tqdm(subset_dataloader, desc=f'Epoch {epoch+1}/{epochs}', unit='batch')):
        try:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs, labels)

        except Exception as e:
            # Si ocurre un error, se salta el batch y se guarda la información
            print(f"Error al procesar el batch {batch_idx} de la epoch {epoch+1}: {e}")
            imagenes_no_identificadas.append((epoch+1, batch_idx))
            continue  # Salta este batch y continúa con el siguiente

    # Promediar la pérdida y la precisión para la época
    avg_loss = running_loss / len(subset_dataloader)
    avg_accuracy = epoch_accuracy / len(subset_dataloader)
    train_losses.append(avg_loss)

    if (epoch + 1) % checkpoint_interval == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }
        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint guardado en {checkpoint_path}")

    print(f"Epoch [{epoch+1}/{epochs}], Pérdida: {avg_loss:.4f}, Precisión: {avg_accuracy:.4f}")

print("Entrenamiento finalizado.")

# Mostrar imágenes no identificadas
if imagenes_no_identificadas:
    print("Imágenes no identificadas durante el entrenamiento:")
    for epoch_num, batch_num in imagenes_no_identificadas:
        print(f"Epoch {epoch_num}, Batch {batch_num}")

# Graficar la pérdida de entrenamiento
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Pérdida de Entrenamiento')
plt.xlabel('Epoch')
plt.ylabel('Pérdida')
plt.title('Pérdida durante el Entrenamiento')
plt.legend()
plt.show()


Epoch 111/150:   1%|          | 4/751 [03:42<11:32:04, 55.59s/batch]


KeyboardInterrupt: 

#continuar desde done guardo


continuar entrenamiento

# dividir el conjunto de datos de prueba y test

In [ ]:
from torch.utils.data import random_split

# Definir el tamaño del conjunto de prueba (10% del dataset)
test_size = int(0.5 * len(dataset))
train_size = len(dataset) - test_size

# Dividir el dataset en entrenamiento y prueba
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Crear DataLoaders para los conjuntos de entrenamiento y prueba
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, prefetch_factor=2)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, prefetch_factor=2)


# Evaluar el Modelo en el Conjunto de Prueba

In [ ]:
import torch
from sklearn.metrics import accuracy_score

def evaluate_model(model, dataloader):
    model.eval()  # Poner el modelo en modo evaluación
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for images, labels in dataloader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(preds.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Evaluar el modelo en el conjunto de datos de prueba
accuracy = evaluate_model(model, test_dataloader)
print(f'Precisión del modelo en el conjunto de prueba: {accuracy:.4f}')


# Predecir Etiquetas para Nuevas Imágenes

In [ ]:
def predict(model, image_path, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Añadir una dimensión de lote
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
    return predicted_class.item()





In [ ]:
# Predecir la clase de una nueva imagen
new_image_path = '/content/drive/MyDrive/IA 2/img_pruebas/dienteLeon2.png'
predicted_class = predict(model, new_image_path, transform)
print(f'La clase predicha es: {predicted_class}')


In [ ]:
# Exportar a TorchScript
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, "mlp_model_scripted.pt")

# Exportar a ONNX
dummy_input = torch.randn(1, 3, 128, 128)
torch.onnx.export(model, dummy_input, "mlp_model.onnx", verbose=True)
